# Notebook exploration

Ce notebook a pour objectif d'explorer de manière superficielle les différents datasets. 


### 1. Importer les différents modules

Cette première étape consiste à charger les librairies nécessaires afin d'explorer 
les datasets.

In [1]:
# Cellule 1 : Import et configuration du chemin
import sys
import os
from pathlib import Path

# On remonte d'un cran pour que le notebook voit la racine du projet
root_path = Path(os.getcwd()).parent
if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

import duckdb
import pandas as pd
from sirene_pipeline.config import settings

# Vérification que la config est bien chargée
print(f"Environnement actuel : {settings.current_env}")
print(f"Dossier Bronze : {settings.bronze_dir}")

Environnement actuel : production
Dossier Bronze : data/bronze


### 2. Inspection générale

La cellule ci-dessous affiche les différentes colonnes des datasets. L'objectif est 
de sélectionner celle que l'on va conserver.

In [3]:
con = duckdb.connect()

# On boucle sur les datasets définis dans ta config pour voir les colonnes de chacun
for dataset_name, config in settings.datasets.items():
    parquet_path = Path(root_path) / "data" / "bronze" / config.filename
    
    print(f"\n--- COLUMNS FOR: {dataset_name} ({config.filename}) ---")
    
    # La commande DESCRIBE donne le nom, le type et si la colonne peut être nulle
    columns_info = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{parquet_path}')").df()
    
    # On affiche tout proprement
    with pd.option_context('display.max_rows', None):
        display(columns_info[['column_name', 'column_type']])


--- COLUMNS FOR: etablissements (stock_etablissement.parquet) ---


,column_name,column_type
0,siren,VARCHAR
1,nic,BIGINT
2,siret,VARCHAR
3,statutDiffusionEtablissement,VARCHAR
4,dateCreationEtablissement,DATE
5,trancheEffectifsEtablissement,VARCHAR
6,anneeEffectifsEtablissement,BIGINT
7,activitePrincipaleRegistreMetiersEtablissement,VARCHAR
8,dateDernierTraitementEtablissement,TIMESTAMP
9,etablissementSiege,BOOLEAN



--- COLUMNS FOR: unites_legales (stock_unite_legale.parquet) ---


,column_name,column_type
0,siren,VARCHAR
1,statutDiffusionUniteLegale,VARCHAR
2,unitePurgeeUniteLegale,BOOLEAN
3,dateCreationUniteLegale,DATE
4,sigleUniteLegale,VARCHAR
5,sexeUniteLegale,VARCHAR
6,prenom1UniteLegale,VARCHAR
7,prenom2UniteLegale,VARCHAR
8,prenom3UniteLegale,VARCHAR
9,prenom4UniteLegale,VARCHAR


**VERDICT:** Après exploration du dataset Bronze, nous avons identifié les colonnes clés pour l'analyse métier et la visualisation. Ces colonnes seront extraites et nettoyées lors du job **Silver**.

---

#### 🏛️ Etablissements (StockEtablissement)
*Focus : Localisation, activité locale et identification des points de vente.*

> **Selected Columns:**
> ```python
> [
>     "siret", "siren", "etatAdministratifEtablissement", 
>     "dateCreationEtablissement", "codePostalEtablissement", 
>     "libelleCommuneEtablissement", "activitePrincipaleEtablissement", 
>     "trancheEffectifsEtablissement", "etablissementSiege", 
>     "enseigne1Etablissement"
> ]
> ```

---

#### ⚖️ Unités Légales (StockUniteLegale)
*Focus : Identité juridique, taille de l'entreprise et santé globale.*

> **Selected Columns:**
> ```python
> [
>     "siren", "denominationUniteLegale", "nomUniteLegale", 
>     "prenom1UniteLegale", "categorieEntreprise", 
>     "categorieJuridiqueUniteLegale", "activitePrincipaleUniteLegale", 
>     "etatAdministratifUniteLegale", "dateCreationUniteLegale", 
>     "economieSocialeSolidaireUniteLegale"
> ]
> ```

---

##### 💾 Mise à jour du `settings.toml`
Les colonnes ont été reportées dans la configuration sous les sections `[silver.etablissements]` et `[silver.unites_legales]`.    